<a href="https://colab.research.google.com/github/Abhijeet8901/Real-Time-Object-Detection-using-YOLOv3/blob/master/Object_detection_with_YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import argparse
import cv2 as cv
import subprocess
import time
import os
from google.colab.patches import cv_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
confidence=0.5
threshold=0.3

In [ ]:
def draw_labels_and_boxes(img, boxes, confidences, classids, idxs, colors, labels):
    
    if len(idxs) > 0:
        for i in idxs.flatten():
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]

            color = [int(c) for c in colors[classids[i]]]

            cv.rectangle(img, (x, y), (x+w, y+h), color, 2)
            text = "{}: {:4f}".format(labels[classids[i]], confidences[i])
            cv.putText(img, text, (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return img

In [ ]:
def generate_boxes_confidences_classids(outs, height, width, tconf):
    boxes = []
    confidences = []
    classids = []

    for out in outs:
        for detection in out:

            scores = detection[5:]
            classid = np.argmax(scores)
            confidence = scores[classid]

            if confidence > tconf:
                
                box = detection[0:4] * np.array([width, height, width, height])
                centerX, centerY, bwidth, bheight = box.astype('int')

               
                x = int(centerX - (bwidth / 2))
                y = int(centerY - (bheight / 2))

               
                boxes.append([x, y, int(bwidth), int(bheight)])
                confidences.append(float(confidence))
                classids.append(classid)

    return boxes, confidences, classids


In [ ]:
def infer_image(net, layer_names, height, width, img, colors, labels,
            boxes=None, confidences=None, classids=None, idxs=None, infer=True):

    if infer:
        
        blob = cv.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
                        swapRB=True, crop=False)

        net.setInput(blob)

        outs = net.forward(layer_names)

        boxes, confidences, classids = generate_boxes_confidences_classids(outs, height, width, confidence)

        idxs = cv.dnn.NMSBoxes(boxes, confidences, confidence, threshold)

    if boxes is None or confidences is None or idxs is None or classids is None:
        raise '[ERROR] You gotta be kidding me.'

    img = draw_labels_and_boxes(img, boxes, confidences, classids, idxs, colors, labels)

    return img, boxes, confidences, classids, idxs

In [ ]:
# Give paths to everything
weights="gdrive/My Drive/yolov3-tiny.weights"
config="gdrive/My Drive/yolov3-tiny.cfg"
video_path="gdrive/My Drive/Chicago_360p.mp4"
label="gdrive/My Drive/coco-labels"
output_path="gdrive/My Drive/output.mp4"

In [ ]:
labels = open(label).read().strip().split('\n')
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')
net = cv.dnn.readNetFromDarknet(config, weights)
layer_names = net.getLayerNames()
layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
vid = cv.VideoCapture(str(video_path))
height, width, writer= None, None, None

while True:

  grabbed, frame = vid.read()
  
  if not grabbed:
	  break

  if width is None or height is None:
    height, width = frame.shape[:2]

  frame, _, _, _, _ = infer_image(net, layer_names, height, width, frame, colors, labels)

  if writer is None:
	  fourcc = cv.VideoWriter_fourcc(*'mp4v')
	  writer = cv.VideoWriter(output_path, fourcc, 30,(frame.shape[1], frame.shape[0]), True)

  writer.write(frame)

writer.release()
vid.release()